In [2]:
import nltk
nltk.download('words')
from nltk.corpus import words
import pandas as pd
import numpy as np
from scipy import stats
import re
import glob
import time
import seaborn
import requests
import pytrends
import datetime as dt
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from urllib.parse import urlparse
import warnings
from bs4 import BeautifulSoup

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Cache-Control': 'no-cache',
    'Pragma': 'no-cache'
}

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\AshleyNguyen\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


## Try the Free API

Get limited features...

In [ ]:
from tqdm import tqdm
import pandas as pd
import json
import requests

# Your existing code
api_key = '8a86af39686e535d9ba6f44889dc0565'
# cb = pd.DataFrame(columns=['CompanyName','UUID', 'Value', 'Permalink', 'Entity Definition ID'])

# Get the total length of the test DataFrame
total_length = len(test)

# Generate a random start position
start_position = 265

# Get a subset of the test DataFrame starting from the random position
subset_test = test.iloc[start_position:]

# Wrap the loop with tqdm
for index, row in tqdm(subset_test.iterrows(), total=len(subset_test), desc="Processing"):
    time.sleep(3)
    company_name = row['CompanyName']
    
    # Encode the company name for the URL
    encoded_company_name = requests.utils.quote(company_name)
    
    # Construct the API URL with the encoded company name
    url = f"https://api.crunchbase.com/api/v4/autocompletes?query={encoded_company_name}&user_key={api_key}"
    
    # Send a GET request to the Crunchbase API
    response = requests.get(url)
    
    try:
        # Attempt to parse the JSON response
        data = json.loads(response.text)
        
        # Extract the first identifier
        if len(data['entities']) != 0:
            identifier = data['entities'][0]['identifier']
            
            # Extract the values from the identifier dictionary
            uuid = identifier['uuid']
            value = identifier['value']
            permalink = identifier['permalink']
            entity_def_id = identifier['entity_def_id']
            
            # Create a temporary DataFrame with the current row's data
            temp_df = pd.DataFrame({'CompanyName': [company_name], 'UUID': [uuid], 'Value': [value], 'Permalink': [permalink], 'Entity Definition ID': [entity_def_id]})
        else:
            temp_df = pd.DataFrame({'CompanyName': [company_name], 'UUID': np.nan, 'Value': np.nan, 'Permalink': np.nan, 'Entity Definition ID': np.nan})
        # Concatenate the temporary DataFrame with the main DataFrame
        cb = pd.concat([cb, temp_df], ignore_index=True)
    except (json.JSONDecodeError, KeyError):
        # Handle JSON decoding error or key error
        print(f"Error occurred while processing {company_name}. Skipping to the next entry.")
        continue

## WebScraper extension to get the Monthly Visits, Acquisition Status, and Operating Status from Crunchbase

In [78]:
cb0 = pd.read_csv("C:\\Users\\AshleyNguyen\\Desktop\\script\\crunchbase\\raw\\cb_5392_5963.csv", header=None)

In [79]:
cb0.head(14)

,0
0,Organization Name
1,Industries
2,Headquarters Location
3,Description
4,CB Rank (Company)
5,Monthly Visits
6,Operating Status
7,Closed Date
8,Website
9,Twitter


In [80]:
cb0.columns

Int64Index([0], dtype='int64')

In [81]:
# Calculate the number of rows and columns in the reshaped table
num_rows = len(cb0) // 14
num_cols = 14

# Reshape the data into a table with 14 columns and 712 rows
reshaped_data = cb0[0].values[:num_rows * num_cols].reshape((num_rows, num_cols))

# Create a new DataFrame from the reshaped data
df_reshaped = pd.DataFrame(reshaped_data[1:], columns=reshaped_data[0])

# Print the new DataFrame
df_reshaped

,Organization Name,Industries,Headquarters Location,Description,CB Rank (Company),Monthly Visits,Operating Status,Closed Date,Website,Twitter,LinkedIn,Founders,Acquisition Status,Acquired by
0,Applied Intuition,"Autonomous Vehicles, Enterprise Software, Infr...","Mountain View, California, United States",Applied Intuition provides software infrastruc...,70,"34,375",Active,—,appliedintuition.com,View on Twitter,View on LinkedIn,"Peter Ludwig, Qasar Younis",Made Acquisitions,—
1,Spiff,"Financial Services, FinTech, SaaS, Sales, Sale...","Sandy, Utah, United States",Spiff is a compensation platform for revenue a...,171,"73,424",Active,—,spiff.com,View on Twitter,View on LinkedIn,"Albert Candari, Casey Crouch, Jean-Philippe Ma...",—,—
2,Strive Health,"Health Care, Information Technology, Medical, ...","Denver, Colorado, United States",Strive Health is a value-based kidney care pro...,174,"10,442",Active,—,strivehealth.com,View on Twitter,View on LinkedIn,Chris Riopelle,—,—
3,Rocket Lab,"Aerospace, Electronics, Industrial Manufacturing","Huntington Beach, California, United States",Rocket Lab delivers a range of complete rocket...,179,"153,959",Active,—,www.rocketlabusa.com,View on Twitter,View on LinkedIn,Peter Beck,Made Acquisitions,—
4,Carrum Health,"Health Care, Hospital, Human Resources, Wellness","San Francisco, California, United States",Carrum Health is a value-based centers of plat...,180,"11,948",Active,—,www.carrumhealth.com,View on Twitter,View on LinkedIn,"Brent Nicholson, Sach Jain",—,—
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,Resonant Sensors Inc.,"Biotechnology, Health Diagnostics, Medical, Se...","Arlington, Texas, United States",The mission of Resonant Sensors Incorporated (...,"1,363,423",—,Active,—,www.resonantsensors.com,View on Twitter,View on LinkedIn,"Debra Weidanz, Robert Magnusson",—,—
511,Feed.fm,"App Marketing, Apps, Media and Entertainment, ...","San Francisco, California, United States","The simplest way to add popular music to apps,...","1,458,561","6,717",Active,—,www.feed.fm,View on Twitter,View on LinkedIn,—,—,—
512,STORES,"Business Development, E-Commerce, E-Commerce P...","Tokyo, Tokyo, Japan",STORES is an e-commerce service platform that ...,"1,468,131",—,Active,—,www.st.inc/,—,—,—,—,—
513,Glidian,"Health Care, Online Portals, Software","South San Francisco, California, United States",Glidian provides authorization solutions to st...,"148,371","3,341",Active,—,www.glidian.com,View on Twitter,View on LinkedIn,"Ashish Dua, Maxwell Wu, Robert Romano",—,—


In [82]:
path = "C:\\Users\\AshleyNguyen\\Desktop\\script\\crunchbase\\"
df_reshaped.to_csv(path+'cb_5392_5963_formatted.csv')

In [83]:
import os
import pandas as pd

path = "C:\\Users\\AshleyNguyen\\Desktop\\script\\crunchbase\\"
output_filename = "crunchbase.csv"

# Get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(path) if file.endswith(".csv")]

# Initialize an empty DataFrame to store the concatenated data
concatenated_data = pd.DataFrame()

# Loop through each CSV file
for file in csv_files:
    # Read the CSV file into a DataFrame
    file_path = os.path.join(path, file)
    data = pd.read_csv(file_path)
    
    # Concatenate the data to the main DataFrame
    concatenated_data = pd.concat([concatenated_data, data])

# Save the concatenated data to a new CSV file
concatenated_data.to_csv(os.path.join(path, output_filename), index=False)


In [124]:
cb = pd.read_csv(path+'crunchbase.csv')

In [125]:
cb.shape

(5146, 15)

In [126]:
cb[cb['Operating Status'] == 'Closed']

,Unnamed: 0,Organization Name,Industries,Headquarters Location,Description,CB Rank (Company),Monthly Visits,Operating Status,Closed Date,Website,Twitter,LinkedIn,Founders,Acquisition Status,Acquired by
205,555,AllCharge.com,"Accounting, Financial Services, Internet, Paym...","Herzliya, Tel Aviv, Israel","AllCharge, an IBM spin-off, is a global Intern...","487,050",393,Closed,—,www.allcharge.com/,—,—,Ilan Zisser,—,—
341,183,Crew,"Apps, Internet, Professional Services","Montréal, Quebec, Canada",Crew is a curated platform that connects desig...,"39,226",624,Closed,—,crew.co/,View on Twitter,View on LinkedIn,"Angus Woodman, Luke Chesser, Mikael Cho, Steph...",Was Acquired,Dribbble
384,397,SecurityMatters,"Cyber Security, Infrastructure, Network Securi...","Eindhoven, Noord-Brabant, The Netherlands",SecurityMatters is a security company that dev...,"54,329",713,Closed,"Nov 8, 2018",www.secmatters.com,View on Twitter,View on LinkedIn,"Damiano Bolzoni, Emmanuele Zambon, Sandro Etalle",Was Acquired,ForeScout Technologies
426,489,Daylife,"Advertising, Brand Marketing, Digital Media, P...","New York, New York, United States",Daylife is a cloud-based media platform offeri...,"290,209",749,Closed,—,www.daylife.com,View on Twitter,View on LinkedIn,"Damon Horowitz, Upendra Shardanand",Was Acquired,Welcome
566,268,BrightFunnel,"B2B, Big Data, Enterprise Software","San Francisco, California, United States","BrightFunnel generates predictive, actionable ...","35,150","1,184",Closed,—,www.brightfunnel.com,View on Twitter,View on LinkedIn,"Nadim Hossain, Nisheeth Ranjan, Ranjan Bagchi",Was Acquired,Terminus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5065,629,Princeton Lightwave,"Manufacturing, Semiconductor","Cranbury, New Jersey, United States",Princeton Lightwave is a manufacturing company...,"323,585",—,Closed,Oct-17,www.princetonlightwave.com,—,—,Nancy Morris,Was Acquired,Argo AI
5080,662,Sleepy Giant,"Digital Entertainment, Gaming, Web Development","Newport Beach, California, United States",Sleepy Giant partners with development teams t...,"456,591",—,Closed,2014,www.sleepygiant.com,View on Twitter,View on LinkedIn,"David Lee, Matthew Hannus",—,—
5083,666,LuckyLabs,"Casual Games, Gaming, Video Games","Boston, Massachusetts, United States",LuckyLabs is an entertainment company developi...,"486,229",—,Closed,—,luckylabs.com,View on Twitter,View on LinkedIn,Steven N. Kane,—,—
5112,712,SkyRecon Systems,"Business Information Systems, Database, Security","Paris, Ile-de-France, France","SkyRecon Systems offers StormShield, an endpoi...","1,085,790",—,Closed,Sep-09,www.skyrecon.com,View on Twitter,View on LinkedIn,"Philippe Honigman, Ravy Truchot",Was Acquired,Stormshield


In [127]:
cb['Acquired by'].value_counts()

—                    4844
Google                  5
Intel                   4
Microsoft               3
Broadcom                2
                     ... 
Applied Intuition       1
Azerion                 1
Rallyware               1
Khan Academy            1
Akana by Perforce       1
Name: Acquired by, Length: 285, dtype: int64

In [128]:
cb['Monthly Visits'] = cb['Monthly Visits'].str.replace(',', '')
cb['Monthly Visits'] = pd.to_numeric(cb['Monthly Visits'], errors='coerce')
cb['Monthly Visits'] = cb['Monthly Visits'].fillna(0)

In [129]:
cb['Monthly Visits'].describe()

count    5.146000e+03
mean     4.923904e+05
std      1.658029e+07
min      0.000000e+00
25%      0.000000e+00
50%      2.317000e+03
75%      1.378800e+04
max      8.855765e+08
Name: Monthly Visits, dtype: float64

In [136]:
cb = cb.sort_values('Monthly Visits', ascending=False)
cb

,Unnamed: 0,Organization Name,Industries,Headquarters Location,Description,CB Rank (Company),Monthly Visits,Operating Status,Closed Date,Website,Twitter,LinkedIn,Founders,Acquisition Status,Acquired by
3579,2,GitHub,"Cloud Computing, Developer Tools, Internet, Pr...","San Francisco, California, United States",GitHub provides code hosting services that all...,76,885576524.0,Active,—,github.com,View on Twitter,View on LinkedIn,"Chris Wanstrath, PJ Hyett, Scott Chacon, Tom P...","Made Acquisitions, Was Acquired",Microsoft
3578,3,Pinterest,"Internet, Social Bookmarking, Social Media, So...","San Francisco, California, United States",Pinterest is a visual bookmarking tool for sav...,141,730149026.0,Active,—,pinterest.com,View on Twitter,View on LinkedIn,"Ben Silbermann, Evan Sharp, Paul Sciarra",Made Acquisitions,—
3577,13,YouAppi,"Advertising, Apps, Big Data, Content, Mobile, ...","Palo Alto, California, United States",YouAppi is a digital marketing platform for pr...,"2,836",274159788.0,Active,—,www.youappi.com,View on Twitter,View on LinkedIn,"Moshe Vaknin, Ofer Garnett",—,—
3576,1,Okta,"CRM, Enterprise Software, Identity Management,...","San Francisco, California, United States",Okta is a management platform that secures cri...,68,136974094.0,Active,—,www.okta.com,View on Twitter,View on LinkedIn,"Frédéric Kerrest, Todd McKinnon",Made Acquisitions,—
3575,112,AfterShip,"Customer Service, E-Commerce, SaaS, Shipping","Tsuen Wan, New Territories, Hong Kong",We help e-commerce businesses grow and make bu...,"5,437",31061492.0,Active,—,www.aftership.com,View on Twitter,View on LinkedIn,Dante Tsang,—,—
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5141,752,logicdialog,"Analytics, Artificial Intelligence, Software","London, England, United Kingdom",logicdialog is a software firm that offers dig...,"2,280,229",0.0,Active,—,www.logicdialog.ai/,View on Twitter,View on LinkedIn,—,—,—
5142,753,Controllis,Energy,"Cambridge, Cambridgeshire, United Kingdom",Controllis provides advanced perpetual power s...,"2,304,643",0.0,Active,—,www.controllis.com/,—,—,—,—,—
5143,754,Beverage Analytics,"Analytics, Apps, Software",—,Beverage Analytics provider of an alcohol anal...,"2,344,166",0.0,Active,—,beverageanalytics.io/,View on Twitter,—,—,—,—
5144,755,The Commerce,"E-Commerce, Marketplace, Shopping","Seoul, Seoul-t'ukpyolsi, South Korea",The Commerce Operator operates an e-commerce p...,"2,529,654",0.0,Active,—,thecommerce.co.kr,—,—,—,—,—


In [137]:
zombie = cb[(cb['Operating Status'] != 'Closed') & (cb['Acquired by'] == '—') & (cb['Monthly Visits'] == 0)]

## News Timeline

In [158]:
directory = r'C:\Users\AshleyNguyen\Desktop\script\crunchbase\news\\'

news = pd.DataFrame()

csv_files = glob.glob(directory + 'news*.csv')

for file in csv_files:
    interest_df = pd.read_csv(file)
    news = pd.concat([news, interest_df], axis = 0)

In [160]:
news['type']

0                                  Friday, June 16, 2023
1      Investments (1) NOVELSAT raised an undisclosed...
2                                Thursday, June 15, 2023
3      Investments (1) SmartKem raised $12,200,000  /...
4                               Wednesday, June 14, 2023
                             ...                        
605    Investments (1) Ogee raised $7,700,000  / Seri...
606                             Wednesday, June 29, 2022
607    News (11)  MakerDAO  Coin Telegraph —  MakerDA...
608                               Tuesday, June 28, 2022
609    News (4)  Prose:  ROI-NJ —  Prose Technologies...
Name: type, Length: 5166, dtype: object

In [162]:
# Calculate the number of rows and columns in the reshaped table
num_rows = len(news) // 2
num_cols = 2

# Reshape the data into a table with 14 columns and 712 rows
reshaped_data = news['type'].values[:num_rows * num_cols].reshape((num_rows, num_cols))

# Create a new DataFrame from the reshaped data
df_reshaped = pd.DataFrame(reshaped_data[1:])

# Print the new DataFrame
df_reshaped

,0,1
0,"Thursday, June 15, 2023","Investments (1) SmartKem raised $12,200,000 /..."
1,"Wednesday, June 14, 2023",News (2) Climate First Bank: Business Journa...
2,"Tuesday, June 13, 2023","Investments (1) Auddia raised $3,100,000 / Po..."
3,"Monday, June 12, 2023",News (1) Auddia: Benzinga — What's Going On...
4,"Thursday, June 8, 2023",News (1) General Graphene: BioSpace — Hemem...
...,...,...
2577,"Monday, July 4, 2022",News (4) MakerDAO: The Defiant — MakerDAO M...
2578,"Friday, July 1, 2022",News (5) Platform Science: Deli Market News ...
2579,"Thursday, June 30, 2022","Investments (1) Ogee raised $7,700,000 / Seri..."
2580,"Wednesday, June 29, 2022",News (11) MakerDAO Coin Telegraph — MakerDA...


In [163]:
df_reshaped = df_reshaped.rename(columns={0: 'date', 1: 'news'})


In [168]:
grouped_df = df_reshaped.groupby('date')['news'].apply(list).reset_index()
grouped_df

,date,news
0,"Friday, April 1, 2022","[Investments (5) HeavyFinance raised €90,000 ..."
1,"Friday, April 14, 2023",[News (1) Forum Systems: PRNewswire — Forum...
2,"Friday, April 15, 2022",[News (3) Noa Botanicals: Business Journals ...
3,"Friday, April 21, 2023",[News (2) Mullen Automotive: Benzinga — Mul...
4,"Friday, April 22, 2022","[Investments (1) Sustaera raised $1,000,000 /..."
...,...,...
559,"Wednesday, October 5, 2022",[Investments (1) Culture Technologies raised €...
560,"Wednesday, September 14, 2022",[News (3) standing ovation: TechCrunch — Fr...
561,"Wednesday, September 21, 2022",[News (1) Danimer Scientific: Tech Times — ...
562,"Wednesday, September 28, 2022",[News (3) Alef Edge: Business Wire — Alef P...


In [169]:
grouped_df.to_csv(path+'news_crunchbase.csv')